In [31]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from joblib import dump,load
from sklearn import metrics


pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(suppress=True)
pd.options.display.max_columns = None

In [32]:
bod = pd.read_csv("../Capstone_project/BOD_test.csv")

In [33]:
bod

,Id,1,2,3,4,5,6,7
0,1,6.80,5.40,NaN,NaN,NaN,NaN,NaN
1,2,4.71,4.20,NaN,NaN,NaN,NaN,NaN
2,32,2.10,3.40,NaN,NaN,NaN,NaN,NaN
3,40,5.35,5.85,NaN,NaN,NaN,NaN,NaN
4,43,4.80,5.30,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
58,206,5.80,6.60,1.75,5.30,15.10,4.60,15.60
59,209,3.68,4.85,NaN,NaN,NaN,NaN,NaN
60,210,7.50,4.80,NaN,NaN,NaN,NaN,NaN
61,215,8.00,6.80,NaN,NaN,NaN,NaN,NaN


In [34]:
bod.isna().sum()

Id     0
1      0
2      0
3     48
4     48
5     47
6     49
7     49
dtype: int64

In [35]:
bod['3'].fillna(bod['3'].mean(), inplace=True)
bod['4'].fillna(bod['4'].mean(), inplace=True)
bod['5'].fillna(bod['5'].mean(), inplace=True)
bod['6'].fillna(bod['6'].mean(), inplace=True)
bod['7'].fillna(bod['7'].mean(), inplace=True)

In [36]:
bod.set_index('Id', inplace=True)

In [37]:
bod

,1,2,3,4,5,6,7
Id,,,,,,,
1,6.80,5.40,4.44,6.05,7.56,4.20,5.19
2,4.71,4.20,4.44,6.05,7.56,4.20,5.19
32,2.10,3.40,4.44,6.05,7.56,4.20,5.19
40,5.35,5.85,4.44,6.05,7.56,4.20,5.19
43,4.80,5.30,4.44,6.05,7.56,4.20,5.19
...,...,...,...,...,...,...,...
206,5.80,6.60,1.75,5.30,15.10,4.60,15.60
209,3.68,4.85,4.44,6.05,7.56,4.20,5.19
210,7.50,4.80,4.44,6.05,7.56,4.20,5.19


In [38]:
bod.dtypes

1    float64
2    float64
3    float64
4    float64
5    float64
6    float64
7    float64
dtype: object

In [39]:
input_cols = bod.columns

In [40]:
input_cols

Index(['1', '2', '3', '4', '5', '6', '7'], dtype='object')

## outlier

In [41]:
model_ee = load("../model_objects/BOD_outlier.joblib")
bod['anomaly_score'] = model_ee.predict(bod[input_cols])
ee_index = list(bod[bod['anomaly_score']==-1].index)

In [42]:
len(ee_index)

4

In [43]:
bod.drop(ee_index, inplace=True)

In [44]:
bod.shape

(59, 8)

In [45]:
del bod['anomaly_score']

In [46]:
bod.head()

,1,2,3,4,5,6,7
Id,,,,,,,
1,6.80,5.40,4.44,6.05,7.56,4.20,5.19
2,4.71,4.20,4.44,6.05,7.56,4.20,5.19
32,2.10,3.40,4.44,6.05,7.56,4.20,5.19
40,5.35,5.85,4.44,6.05,7.56,4.20,5.19
43,4.80,5.30,4.44,6.05,7.56,4.20,5.19


In [47]:
sc = load("../model_objects/BOD_standard.joblib")
sc_bod = pd.DataFrame(sc.transform(bod[input_cols]),columns=input_cols,index=bod.index)
sc_bod.head()

,1,2,3,4,5,6,7
Id,,,,,,,
1,0.88,0.26,-0.23,0.39,-0.68,0.46,1.86
2,-0.13,-0.36,-0.23,0.39,-0.68,0.46,1.86
32,-1.39,-0.77,-0.23,0.39,-0.68,0.46,1.86
40,0.18,0.49,-0.23,0.39,-0.68,0.46,1.86
43,-0.08,0.21,-0.23,0.39,-0.68,0.46,1.86


In [48]:
bod.drop(columns = input_cols, inplace=True)

In [49]:
bod = pd.concat([bod,sc_bod], axis=1)
bod

,1,2,3,4,5,6,7
Id,,,,,,,
1,0.88,0.26,-0.23,0.39,-0.68,0.46,1.86
2,-0.13,-0.36,-0.23,0.39,-0.68,0.46,1.86
32,-1.39,-0.77,-0.23,0.39,-0.68,0.46,1.86
40,0.18,0.49,-0.23,0.39,-0.68,0.46,1.86
43,-0.08,0.21,-0.23,0.39,-0.68,0.46,1.86
46,-0.25,-0.61,-0.23,0.39,-0.68,0.46,1.86
53,0.22,-0.27,-0.23,0.39,-0.68,0.46,1.86
60,-0.49,-0.59,3.17,1.52,-2.92,0.46,1.86
61,-0.06,-0.72,-0.23,0.39,-0.68,0.46,1.86


## Predictions

In [50]:
best_model = load("../model_objects/BOD_best_model.joblib")

In [51]:
pred = best_model.predict(bod)

In [52]:
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0])

In [53]:
bod['target'] = pred

In [29]:
bod

,1,2,3,4,5,6,7,target
Id,,,,,,,,
1,0.88,0.26,-0.23,0.39,-0.68,0.46,1.86,1
2,-0.13,-0.36,-0.23,0.39,-0.68,0.46,1.86,0
32,-1.39,-0.77,-0.23,0.39,-0.68,0.46,1.86,0
40,0.18,0.49,-0.23,0.39,-0.68,0.46,1.86,0
43,-0.08,0.21,-0.23,0.39,-0.68,0.46,1.86,0
46,-0.25,-0.61,-0.23,0.39,-0.68,0.46,1.86,0
53,0.22,-0.27,-0.23,0.39,-0.68,0.46,1.86,0
60,-0.49,-0.59,3.17,1.52,-2.92,0.46,1.86,0
61,-0.06,-0.72,-0.23,0.39,-0.68,0.46,1.86,0


In [54]:
bod['target'].value_counts()

0    50
1     9
Name: target, dtype: int64